# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 21

### Elementos do Grupo

Número: 56926    Nome: Lucas Pinta    
Número: 56895    Nome: Matilde Silva    
Número: 56941    Nome: Bruno Gonzaez

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

In [74]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente

# Um estado é representado pelos valores dinâmicos (e mínimos), portanto optámos por representar
# estes pela posição do pacman, as pastilhas disponíveis, os pontos, e as células visitadas para
# posteriormente calcular o custo. Isto foi feito através da classe PacmanEstado

import time as t
from typing import Tuple

class PacmanEstado:
    def __init__(self, pacman=(1, 1), gums=None, cellsVisited=None, points=0):

        self.pacman    = pacman
        self.gums = gums
        self.points    = points

        if gums is None:
            gums = {}

        if cellsVisited is None:
            self.cellsVisited = {pacman: 1}
        else:
            self.cellsVisited = cellsVisited
        """ cellsVisited
            Key: (x,y) of visited cell
            Value: How many times it was visited
        """

    def __lt__(self, other: PacmanEstado) -> bool:
        if not isinstance(other, PacmanEstado):
            # don't attempt to compare against unrelated types
            return NotImplemented
            
        return self.points < other.points

    def __eq__(self, other: PacmanEstado) -> bool:
        if not isinstance(other, PacmanEstado):
            # don't attempt to compare against unrelated types
            return NotImplemented

        return self.pacman == other.pacman\
                and self.points == other.points\
                and self.gums == other.gums\
                and self.cellsVisited == other.cellsVisited
    
    def visitCell(self, cell: Tuple[int, int], start: int):
        """Visita a célula e altera o custo"""
        if cell not in self.cellsVisited:
            self.cellsVisited[cell] = 0        

        self.cellsVisited[cell] += 1

        if cell in self.gums:
            self.eatGum(cell, start)

    def eatGum(self, gum: Tuple[int, int], start: int):
        """"Come a pastilha e altera o valor da pontuação"""
        if self.gums[gum] == 'N':
            self.points += 1
        elif self.gums[gum] == 'D':
            self.points += max(0, 5-(t.time()-start))
        elif self.gums[gum] == 'C':
            self.points += t.time()-start

        del self.gums[gum]

In [ ]:
# representação de um estado à sua escolha
"""
pacman: (2, 1)
pastilhas: {(5, 8): 'N', (7, 3): 'C', (4, 5): 'D'}
points: 1
cellsVisited: {(1, 1): 1, (2, 1): 1}
"""

# Desenhado, ficaria da seguinte forma:
"""
= = = = = = = = = = 
= . @ . . . . . . = 
= . = = = = = = . = 
= . = . . . = C . = 
= . = . . . = . . = 
= . = . D . = . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . N . . . = 
= = = = = = = = = = 
"""

## Formulação do problema

Para a organização de dados criou-se a classe "PacmanPastilhas" e o seu construtor recebe os parâmetros:
- **pacman** — Tuplo (x,y) com a posição do Pacman
- **goal** —  Int do pontos-objetivo
- **gums** — Dicionário contendo chaves associadas a posições do mapa e respetivos valores contendo o seu custo atual
- **obstacles** — Dicionário com as posições (x e y) que o Pacman não pode utilizar
- **dim** — Int da dimensão do mapa

para além destes, também são inicializados os pârametros:
- **timeStart** — Inicialização do contador de tempo utilizado nas pastilhas que requerem este valor
- **directions** — Dicionário em que as chaves são os movimentos e os valores a respetiva soma ou subtração a aplicar nas coordenadas atuais

Esta classe possui funções:
- **actions(self, state: PacmanEstado)** — Verifica a posição dada e verifica quais são as possibilidades a partir deste ponto
- **result(self, state: PacmanEstado, action: str)** — Com base no estado atual e dada ação, altera o estado do pacman
- **goal_test(self, state: PacmanEstado)** — Compara os pontos atuais com o objetivo para se determinar se o objetivo já foi alcançado.
- **path_cost(self, c: int, state1: PacmanEstado, action: str, state2: PacmanEstado)** — O método recebe como parâmetro o custo acumulado do estado inicial até state1 (c) e adiciona-lhe o custo de ir de state1 para state2.
- **exec(self, state: PacmanEstado, actions: List[str])** — Recurrendo a funções anteriores (result(),path_cost,goal_test()) para calcular todo o custo que foi gerado no desenrolar do algoritmo até ao ponto que se pretende saber.
- **display(self, state: PacmanEstado)** — Representação usando caracteres do mapa em determinado estado mostrando o pacman, pastilhas e obstáculo

In [76]:
from searchPlus import *
import time as t
from typing import Tuple, List

class PacmanPastilhas(Problem):
    def __init__(self, pacman=(1, 1), goal=1, gums={}, obstacles={}, dim=10):
        super().__init__(PacmanEstado(pacman, gums), goal)
        self.dim = dim
        self.obstacles = obstacles
        self.timeStart = t.time()
        self.directions = {"N":(0, -1), "W":(-1, 0), "E":(1, 0),"S":(0, 1)}  

    def actions(self, state: PacmanEstado) -> List[str]:
        """
        Retorna as ações que podem ser executadas para um dado estado.
        """
        x, y = state.pacman
        return [act for act in self.directions
                if (x + self.directions[act][0], y + self.directions[act][1]) not in self.obstacles]

    def result(self, state: PacmanEstado, action: str) -> PacmanEstado:
        """
        Retorna o estado que resulta de executar uma dada ação num 
        dado estado. A ação deve ser uma das self.actions(state).
        """
        x, y = state.pacman
        dx, dy = self.directions[action]

        e = PacmanEstado((x+dx, y+dy), state.gums, state.cellsVisited, state.points)
        e.visitCell(e.pacman, self.timeStart)
        return e

    def goal_test(self, state: PacmanEstado) -> bool:
        """
        Retorna True se o estado que atingiu é o objetivo.
        """
        return state.points >= self.goal

    def path_cost(self, c: int, state1: PacmanEstado, action: str, state2: PacmanEstado) -> int:
        """
        Retorna o custo de uma solução que chega ao state2 através
        do state1, assumindo custo acumlado c de state1.
        """
        # c -> custo até de todo o caminho até state1
        # state1 -> estado antgo
        # state2 -> estado novo #SALAZAR 17499 dias
        # action -> ação de state1 para state2

        #state2 = self.result(state1,action)

        # ir ao dicionario ver qt custa state2 e
        # soma ao custo que está associado a state1 
        
        return c + state2.cellsVisited[state2.pacman]
        
    def exec(self, state: PacmanEstado, actions: List[str]) -> Tuple[PacmanEstado, int]:
        """
        Tuplo com o estado atual do Pacman e o respetivo custo acumulado até esta posição
        """
        custo = 0
        for a in actions:
            seg = self.result(state, a)
            custo = self.path_cost(custo, state, a, seg)
            state = seg
        self.display(state)
        print('Custo:', custo)
        print('Goal?', self.goal_test(state))
        return (state, custo)

    def display(self, state: PacmanEstado):
        """
        Constrói o mapa 2D da representação de um estado
        """
        grid = ''
        for y in range(self.dim):
            for x in range(self.dim):
                if (x, y) in self.obstacles:
                    grid += '= '
                elif (x, y) == state.pacman:
                    grid += '@ '
                elif (x,y) in state.gums:
                    grid += f'{state.gums[(x, y)]} '
                else:
                    grid += '. '
            grid += '\n'

        print(grid, end='')


## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [77]:
# código de teste do problema
from random import randint

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x, y, 0, 1, length)\
        | line(x + length - 1, y, 0, 1, length)\
        | line(x, y, 1, 0, length)\
        | line(x, y + length - 1, 1, 0, length)

def mostra(custo, final, acoes, acao):
    """Mostra e formata a mensagem."""
    print(f'Custo: {custo}')
    print(f'É final? {"Sim" if final else "Não"}')
    print(f'Ações: {acoes}. Escolhi {acao}')

l = line(2, 2, 1, 0, 6)
c = line(2, 3, 0, 1, 4)
d = line(6, 3, 0, 1, 3)
fronteira = quadro(0, 0, 10)

prob = PacmanPastilhas(
    pacman=(1, 1),
    goal=1,
    gums={(2,1): 'N', (5, 8): 'N', (7, 3): 'C', (4,5): 'D'},
    obstacles=fronteira | l | c | d,
    dim=10)

est = prob.initial
prob.display(est)
final = prob.goal_test(est)
acoes = prob.actions(est)
acao = acoes[randint(0, len(acoes)-1)]
custo = 0
mostra(custo, final, acoes, acao)
print('------------------------')

# Mostrar estados até chegar ao objetivo ou 500 iterações
for i in range(500):
    novo_est = prob.result(est, acao)
    prob.display(novo_est)
    final = prob.goal_test(novo_est)
    acoes = prob.actions(novo_est)
    acao = acoes[randint(0, len(acoes)-1)]
    # Não precisamos do estado anterior nem da acao
    custo = prob.path_cost(custo, est, None, novo_est)
    mostra(custo, final, acoes, acao)
    print(f'São o mesmo estado? {"Sim" if est == novo_est else "Não"}')
    print(f'O estado anterior é menor? {"Sim" if est < novo_est else "Não"}')
    print('------------------------')

    if final:
        break
    est = novo_est


= = = = = = = = = = 
= @ N . . . . . . = 
= . = = = = = = . = 
= . = . . . = C . = 
= . = . . . = . . = 
= . = . D . = . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . N . . . = 
= = = = = = = = = = 
Custo: 0
É final? Não
Ações: ['E', 'S']. Escolhi S
------------------------
= = = = = = = = = = 
= . N . . . . . . = 
= @ = = = = = = . = 
= . = . . . = C . = 
= . = . . . = . . = 
= . = . D . = . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . N . . . = 
= = = = = = = = = = 
Custo: 1
É final? Não
Ações: ['N', 'S']. Escolhi S
São o mesmo estado? Não
O estado anterior é menor? Não
------------------------
= = = = = = = = = = 
= . N . . . . . . = 
= . = = = = = = . = 
= @ = . . . = C . = 
= . = . . . = . . = 
= . = . D . = . . = 
= . = . . . . . . = 
= . . . . . . . . = 
= . . . . N . . . = 
= = = = = = = = = = 
Custo: 2
É final? Não
Ações: ['N', 'S']. Escolhi S
São o mesmo estado? Não
O estado anterior é menor? Não
------------------------
= = = = = = = = = = 
= . N . . .

## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [ ]:
# código de aplicação dos algoritmos